### Debt Repayment Plan

#### Part 2 - Advanced Case

- Data
    - **N<sub>i</sub>**  : Monthly income during period **i**
    - **E<sub>i</sub>**  : Monthly expense during period **i**
    - **M<sub>i</sub>**  : Monthly required savings during period **i**
    - **I**              : Interest paid on loan
    

- Decision Variables
    - **S<sub>i</sub>**  : Monthly savings during period **i**
    - **R<sub>i</sub>**  : Debt paid during month **i**
    - **X<sub>i</sub>**  : Debt owed at the beginning of month **i**

In [1]:
using JuMP
import HiGHS